In [1]:
from crawl import Crawler
from parse import find_all_safely
import pandas as pd
import numpy as np
import os
from setup_db import DB, setup_db, cursor_as_df, execute_to_df

In [2]:
# db = setup_db()

In [3]:
path = "test_db.db"
db = DB(path)
db.run_setup_scripts()
#db.insert_initial_data()


In [ ]:
# db.close()

In [4]:
db.select_all("jobs")

,job_id,user_created,active,time_created,last_run,job_name
0,1,1,1,2019-10-10,None,test_job
1,2,1,1,2019-10-11,None,test2_job
2,3,1,1,2020-01-02 20:01:20,None,trip to Cracow
3,6,2,1,2020-01-02 21:20:48,None,Trip to the sea in March
4,7,2,1,2020-01-02 21:24:43,None,City break in January
5,8,1,1,2020-01-02 21:30:42,None,Trip between Gdansk and Lodz


In [5]:
db.select_all("requests")

,request_id,job_id,start_city,end_city,date
0,1,1,1915,7588,2020-01-02
1,2,1,1915,7588,2020-01-03
2,3,1,1915,7588,2020-01-04
3,4,2,1915,1815,2020-01-02
4,5,3,7588,1915,2020-01-20
5,6,3,7588,1915,2020-01-21
6,7,3,7588,1915,2020-01-22
7,8,3,7588,1915,2020-01-23
8,9,3,7588,1915,2020-01-24
9,10,3,7588,1915,2020-01-25


In [6]:
db.show_tables()

[('cities',),
 ('distances',),
 ('execution_logs',),
 ('jobs',),
 ('sqlite_sequence',),
 ('requests',),
 ('results',),
 ('users',),
 ('working_days',)]

In [7]:
db.select_all("results")

,result_id,request_id,time_created,start_city,end_city,time,date,price,changes_number
0,1,1,1577728921,1915,7588,00:30,02.01.2020,69.99,0
1,2,1,1577728980,1915,7588,07:05,02.01.2020,69.99,0
2,3,1,1577729000,1915,7588,12:00,02.01.2020,69.99,0
3,4,1,1577729242,1915,7588,15:10,02.01.2020,69.99,0
4,5,1,1577729540,1915,7588,19:15,02.01.2020,69.99,0
5,6,1,1577730200,1915,7588,23:59,02.01.2020,39.99,0
6,7,2,1577730280,1915,7588,00:30,03.01.2019,69.99,0
7,8,2,1577730350,1915,7588,07:05,03.01.2019,55.99,0
8,9,2,1577730700,1915,7588,12:00,03.01.2019,59.99,0
9,10,2,1577730956,1915,7588,15:10,03.01.2019,69.99,0


SELECT 
request_id,
strftime('%d.%m.%Y',t2.time_created) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1


In [8]:
params_from_db = db.c.execute('''SELECT 
t2.request_id,
strftime("%d.%m.%Y",t2.date) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests as t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1''')
colnames = [i[0] for i in params_from_db.description]
values = params_from_db.fetchall()

In [9]:
print(colnames)
print(values)

['request_id', 'rideDate', 'departureCity', 'arrivalCity']
[(1, '02.01.2020', 1915, 7588), (2, '03.01.2020', 1915, 7588), (3, '04.01.2020', 1915, 7588), (4, '02.01.2020', 1915, 1815), (5, '20.01.2020', 7588, 1915), (6, '21.01.2020', 7588, 1915), (7, '22.01.2020', 7588, 1915), (8, '23.01.2020', 7588, 1915), (9, '24.01.2020', 7588, 1915), (10, '25.01.2020', 7588, 1915), (11, '26.01.2020', 7588, 1915), (12, '27.01.2020', 7588, 1915), (13, '28.01.2020', 7588, 1915), (14, '29.01.2020', 7588, 1915), (15, '30.01.2020', 7588, 1915), (16, '31.01.2020', 7588, 1915), (17, '01.02.2020', 7588, 1915), (18, '02.03.2020', 7588, 1815), (19, '03.03.2020', 7588, 1815), (20, '04.03.2020', 7588, 1815), (21, '05.03.2020', 7588, 1815), (22, '18.01.2020', 7588, 1915), (23, '15.01.2020', 1815, 7588), (24, '16.01.2020', 1815, 7588), (25, '17.01.2020', 1815, 7588), (26, '18.01.2020', 1815, 7588), (27, '19.01.2020', 1815, 7588), (28, '20.01.2020', 1815, 7588)]


In [10]:
constant_search_params = {
    "adult": "1",
    "_locale": "pl"
}


In [11]:
values

[(1, '02.01.2020', 1915, 7588),
 (2, '03.01.2020', 1915, 7588),
 (3, '04.01.2020', 1915, 7588),
 (4, '02.01.2020', 1915, 1815),
 (5, '20.01.2020', 7588, 1915),
 (6, '21.01.2020', 7588, 1915),
 (7, '22.01.2020', 7588, 1915),
 (8, '23.01.2020', 7588, 1915),
 (9, '24.01.2020', 7588, 1915),
 (10, '25.01.2020', 7588, 1915),
 (11, '26.01.2020', 7588, 1915),
 (12, '27.01.2020', 7588, 1915),
 (13, '28.01.2020', 7588, 1915),
 (14, '29.01.2020', 7588, 1915),
 (15, '30.01.2020', 7588, 1915),
 (16, '31.01.2020', 7588, 1915),
 (17, '01.02.2020', 7588, 1915),
 (18, '02.03.2020', 7588, 1815),
 (19, '03.03.2020', 7588, 1815),
 (20, '04.03.2020', 7588, 1815),
 (21, '05.03.2020', 7588, 1815),
 (22, '18.01.2020', 7588, 1915),
 (23, '15.01.2020', 1815, 7588),
 (24, '16.01.2020', 1815, 7588),
 (25, '17.01.2020', 1815, 7588),
 (26, '18.01.2020', 1815, 7588),
 (27, '19.01.2020', 1815, 7588),
 (28, '20.01.2020', 1815, 7588)]

In [12]:
params_list = []
for single_params in values:
    params_list.append({"rideDate": single_params[1],
                       "departureCity": single_params[2],
                       "arrivalCity": single_params[3]})
    
params_list = [{**i, **constant_search_params} for i in params_list]
request_id_list = [i[0] for i in values]

In [13]:
params_list

[{'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '03.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '04.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 1815,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '20.01.2020',
  'departureCity': 7588,
  'arrivalCity': 1915,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '21.01.2020',
  'departureCity': 7588,
  'arrivalCity': 1915,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '22.01.2020',
  'departureCity': 7588,
  'arrivalCity': 1915,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '23.01.2020',
  'departureCity': 7588,
  'arrivalCity': 1915,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '24.01.2020',
  'departureCity': 7588,
  'arrivalCity': 1915,
  'adult': '1',
  '_locale':

In [14]:
current_params = params_list[1]
current_request_id = request_id_list[1]

In [15]:
crawler = Crawler(current_params)
crawler.get_all()
results_pd_raw  = crawler.get_pandas_df()

In [16]:
results_pd = results_pd_raw.copy()

In [17]:
results_pd_raw

,departure_time,arrival_time,departure_station,arrival_station,price
0,00:30,04:00,Kraków MDA,Łódź Fabryczna,"69,99"
1,07:05,11:25,Kraków MDA,Łódź Fabryczna,"59,99"
2,12:00,16:00,Kraków MDA,Łódź Fabryczna,"59,99"
3,15:10,19:25,Kraków MDA,Łódź Fabryczna,"69,99"
4,19:15,23:50,Kraków MDA,Łódź Fabryczna,None
5,23:59,04:15,Kraków MDA,Łódź Fabryczna,"49,99"
6,00:30,04:00,Kraków MDA,Łódź Fabryczna,"59,99"


In [ ]:
class dfSanitizer:
    def __init__(self, df):
        self.df = df

#         self.sanitize_station_vec = np.vectorize(sanitize_station)
        self.sanitize_price_vec = np.vectorize(sanitize_price)
#         self.sanitize_time_vec = np.vectorize(sanitize_time)
    
    def sanitize_price(self, x):
        if x is None:
            return None

        try:
            stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas

            if len(stripped)>10:
                return None

            stripped_comma = stripped.replace(",", ".")

            number = float(stripped_comma)

        except:
            return None

        return number
    

In [ ]:
a = dfSanitizer(1)
a

In [ ]:
a.sanitize_price_vec(10)

In [ ]:
def sanitize_price(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas
        
        if len(stripped)>10:
            return None

        stripped_comma = stripped.replace(",", ".")
        
        number = float(stripped_comma)
    
    except:
        return None
    
    return number
    

In [ ]:
print(sanitize_price("65,94") == 65.94)
print(sanitize_price("65,a94") == 65.94)
print(sanitize_price("65,a94   ") == 65.94)
print(sanitize_price("65,56,4") is None)
print(sanitize_price(None) is None)
print(sanitize_price("54,") == 54.0)
print(sanitize_price(",54") == 0.54)
print(sanitize_price(43) is None)

In [ ]:
def sanitize_station(station):
    try:
        return station.lstrip().rstrip()
    except:
        return None

In [ ]:
print(sanitize_station("abc") == "abc")
print(sanitize_station("      abc") == "abc")
print(sanitize_station("abc   ") == "abc")
print(sanitize_station(1) is None)
print(sanitize_station(None) is None)

In [ ]:
def sanitize_time(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\:]", "", x) # remove all characters except numbers and commas
        
        if len(stripped) != 5:
            return None

        if sum([i == ":" for i in stripped]) != 1:
            return None
        
    
    except:
        return None
    
    return stripped

In [ ]:
print(sanitize_time("09:50") == "09:50")
print(sanitize_time("09:50   ") == "09:50")
print(sanitize_time("09:50aa") == "09:50")
print(sanitize_time("9:50:") is None)
print(sanitize_time("950") is None)
print(sanitize_time(":") is None)
print(sanitize_time("9:50") is None)

In [ ]:

sanitize_station_vec = np.vectorize(sanitize_station)
sanitize_price_vec = np.vectorize(sanitize_price)
sanitize_time_vec = np.vectorize(sanitize_time)

In [ ]:
results_pd

In [ ]:
results_pd.price = sanitize_price_vec(results_pd.price)
results_pd.departure_station = sanitize_station_vec(results_pd.departure_station)
results_pd.arrival_station = sanitize_station_vec(results_pd.arrival_station)
results_pd.departure_time = sanitize_time_vec(results_pd.departure_time)
results_pd.arrival_time = sanitize_time_vec(results_pd.arrival_time)
results_pd["fully_booked"] = results_pd.price.isna()

In [ ]:
results_pd["rideDate"] = current_params["rideDate"]
results_pd["departure_datetime"] = results_pd.rideDate + " " + results_pd.departure_time
results_pd["departure_datetime"] = pd.to_datetime(results_pd["departure_datetime"])
results_pd["arrival_datetime"] = results_pd.rideDate + " " + results_pd.arrival_time
results_pd["arrival_datetime"] = pd.to_datetime(results_pd["arrival_datetime"])


In [ ]:
mask = results_pd["arrival_datetime"] < results_pd["departure_datetime"]
masked_df = results_pd[mask]

In [ ]:
results_pd.loc[mask, "arrival_datetime"] = masked_df.arrival_datetime + pd.DateOffset(days = 1)

In [ ]:
from datetime import datetime

In [ ]:
datetime.now()

In [ ]:
current_params

In [ ]:
results_pd["request_id"] = current_request_id
results_pd["time_created"] = datetime.now()
results_pd["departure_city"] = current_params["departureCity"]
results_pd["arrival_city"] = current_params["arrivalCity"]

In [ ]:
db.select_all("results")

In [ ]:

results_pd

In [ ]:
unique_stations = set(results_pd.departure_station.tolist() + results_pd.arrival_station.tolist())
unique_stations

In [ ]:
results = []
for search_params in params_list:
    crawler = Crawler(search_params)
    crawler.get_all()
    results.append(crawler.get_pandas_df())
    
results

In [3]:
sanitizer.sanitized_df

,departure_time,arrival_time,departure_station,arrival_station,price,fully_booked,departure_city,arrival_city,request_id
0,23:59,04:15,Kraków MDA,Łódź Fabryczna,69.99,False,1915,7588,1
1,00:30,04:00,Kraków MDA,Łódź Fabryczna,69.99,False,1915,7588,1


In [ ]:
sanitizer